In [ ]:
import numpy as np
import cv2
import xraylib
import matplotlib.pyplot as plt
from holotomocupy.utils import *
from holotomocupy.holo import *

In [ ]:
n = 512  # object size in each dimension
energy = 33.35  # [keV] xray energy
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
ndist = 4
distances = np.array([0.0029432,0.00306911,0.00357247,0.00461673])[:ndist] # [m]
magnification = 400
detector_pixelsize = 3.03751e-6
voxelsize = detector_pixelsize/magnification*2048/n  # object voxel size

ne = n+n//4
show = True


In [ ]:
img = np.zeros((ne, ne, 3), np.uint8)
c=0.5
triangle = np.array([(ne//16/c, ne//2-ne//32), (ne//16/c, ne//2+ne//32), (ne//2-ne//128, ne//2)], np.float32)
star = img[:,:,0]*0
for i in range(0, 360, 15):
    img = np.zeros((ne, ne, 3), np.uint8)
    degree = i
    theta = degree * np.pi / 180
    rot_mat = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]], np.float32)    
    rotated = cv2.gemm(triangle-ne//2, rot_mat, 1, None, 1, flags=cv2.GEMM_2_T)+ne//2
    cv2.fillPoly(img, [np.int32(rotated)], (255, 0, 0))
    star+=img[:,:,0]
[x,y] = np.meshgrid(np.arange(-ne//2,ne//2),np.arange(-ne//2,ne//2))
x = x/ne*2
y = y/ne*2
# add holes in triangles
circ = (x**2+y**2>0.145)+(x**2+y**2<0.135)
circ *= (x**2+y**2>0.053)+(x**2+y**2<0.05)
circ *= (x**2+y**2>0.0085)+(x**2+y**2<0.008)
circ *= (x**2+y**2>0.52)+(x**2+y**2<0.5)

star = star*circ/255

# star[ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2]=0

v = np.arange(-ne//2,ne//2)/ne
[vx,vy] = np.meshgrid(v,v)
v = np.exp(-5*(vx**2+vy**2))
fu = np.fft.fftshift(np.fft.fftn(np.fft.fftshift(star)))
star = np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(fu*v))).real

delta = 1-xraylib.Refractive_Index_Re('Au',energy,19.3)
beta = xraylib.Refractive_Index_Im('Au',energy,19.3)

thickness = 2e-6/voxelsize # siemens star thickness in pixels
# form Transmittance function
u = star*(-delta+1j*beta) # note -delta
Ru = u*thickness 
psi = np.exp(1j * Ru * voxelsize * 2 * np.pi / wavelength)[np.newaxis].astype('complex64')
fig, axs = plt.subplots(1, 2, figsize=(9, 4))
im=axs[0].imshow(np.abs(psi[0]),cmap='gray')
axs[0].set_title('amplitude')
fig.colorbar(im)
im=axs[1].imshow(np.angle(psi[0]),cmap='gray')
axs[1].set_title('phase')
fig.colorbar(im)

mshow_polar(psi[0,ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2],show)


In [ ]:
!wget -nc https://g-110014.fd635.8443.data.globus.org/holotomocupy/examples_synthetic/data/prb_id16a/prb_abs_2048.tiff -P ../data/prb_id16a
!wget -nc https://g-110014.fd635.8443.data.globus.org/holotomocupy/examples_synthetic/data/prb_id16a/prb_phase_2048.tiff -P ../data/prb_id16a

import scipy.ndimage as ndimage
prb_abs = dxchange.read_tiff(f'../data/prb_id16a/prb_abs_2048.tiff')[0:1]
prb_phase = dxchange.read_tiff(f'../data/prb_id16a/prb_phase_2048.tiff')[0:1]
prb = prb_abs*np.exp(1j*prb_phase).astype('complex64')

prb = ndimage.zoom(prb,(1,2*n/2048,2*n/2048))

prb /= np.mean(np.abs(prb))
# prb[:]=1

fig, axs = plt.subplots(1, 2, figsize=(9, 4))
im=axs[0].imshow(np.abs(prb[0]),cmap='gray')
axs[0].set_title('abs prb')
fig.colorbar(im)
im=axs[1].imshow(np.angle(prb[0]),cmap='gray')
axs[1].set_title('angle prb')
fig.colorbar(im)

In [ ]:
from scipy.io import savemat,loadmat
savemat('psi.mat',{'psi': psi[0]})

In [ ]:
np.loadtxt('data2.mat')

In [ ]:
def Lop(psi):
    data = cp.zeros([1,ndist, n, n], dtype='complex64')
    for i in range(ndist):
        ff = cp.array(psi)            
        fx = cp.fft.fftfreq(2*ne, d=voxelsize).astype('float32')
        [fx, fy] = cp.meshgrid(fx, fx)
        fP = cp.exp(-1j*cp.pi*wavelength*distances[i]*(fx**2+fy**2))
        ff = cp.pad(ff,((0,0),(ne//2,ne//2),(ne//2,ne//2)))
        ff = cp.fft.ifft2(cp.fft.fft2(ff)*fP)
        data[:, i] = ff[:,ne-n//2:ne+n//2,ne-n//2:ne+n//2]
    return data
print(voxelsize)


In [ ]:
psi = cp.array(psi)
data = np.abs(Lop(psi))**2
ref = np.abs(Lop((psi*0+1)))**2

for k in range(ndist):
    mshow(data[0,k],show)

savemat('data.mat',{'data': data[0]})
